# **Food-101 Image Classification**

## **Installation and import of libraries**

In [84]:
pip install mlflow 

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [85]:
!pip install mlflow dagshub

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [86]:
pip install dagshub

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [87]:
pip install great-expectations

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
## Import all libraries needed
import os
import ast
import shutil
import urllib
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import mlflow
import mlflow.pytorch
import dagshub
import great_expectations as gx
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib
import matplotlib.pyplot as plt

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision.io import read_image
from torchvision.transforms.functional import to_pil_image
from great_expectations.dataset import PandasDataset
from tqdm import tqdm
from typing import Tuple, Dict, Any, List
from PIL import Image
from typing import Tuple, List
from matplotlib.pyplot import imshow

torch.set_grad_enabled(True)
%matplotlib inline

/usr/local/lib/python3.9/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [2]:
dagshub.init("taed2-Food_Classification", "violeta51", mlflow=True)

mlflow.set_tracking_uri('https://dagshub.com/violeta51/taed2-Food_Classification.mlflow')
mlflow.set_experiment(experiment_name="REPORT INICIAL")

Repository initialized!

<Experiment: artifact_location='mlflow-artifacts:/62f4f6b488564b148a8d5dcc870cb0e6', creation_time=1696057452443, experiment_id='2', last_update_time=1696057452443, lifecycle_stage='active', name='REPORT INICIAL', tags={}>

## **Initialization**

Let's set the working enviroment.

In [3]:
# set a seed for garantee the replication once we finish get the model

seed = 7767
np.random.seed(seed)
_ = torch.manual_seed(seed)
_ = torch.cuda.manual_seed(seed)

Since the model is fairly complex, training it in the GPU will save us time.

In [4]:
# Let's define some hyper-parameters

hparams = {
    'log_interval': 200,
    'epochs' : 20,
    'batch_size' : 64,
}

mlflow.log_param("log_interval", hparams["log_interval"])
mlflow.log_param("epochs", hparams["epochs"])
mlflow.log_param("batch_size", hparams["batch_size"])

64

## **Load the Data**
Since the training data are off-line, we need to load them.

In [5]:
## img dir: directory containing the images
## annotations file: file containing the labels

## the following code is a modified version of 
## the code extracted from https://pytorch.org/tutorials/beginner/basics/data_tutorial.html 

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = annotations_file
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self): # returns the number of samples in our dataset.
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        """if len(img_path)< 6:
            print(img_path)"""
            
        image = read_image(img_path)

        to_pil = transforms.ToPILImage()
        image = to_pil(image)
        
        width, height = image.size
        
        if width < 224 or height < 224:
            scale_factor = 512 / max(width, height)
    
        # calculate the new dimensions
            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)

            image = image.resize((new_height, new_width))
        
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        
        return image, label

In [6]:
# define the composed transformation for training
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224), transforms.RandomHorizontalFlip(p=0.5), transforms.ToTensor(),])

In [7]:
## read the folders and load them in a specific class structure

first = False
first_first = True

# transform the target 'words' into numerical values, this dictionary will associate these two values
dictionary = {} 
idx = 0

for dirname, _, filenames in os.walk('/Users/violeta/Desktop/gced/Q7/TAED2/project1/taed2-Food_Classification/data/processed'):

    if first == True:
        label = dirname[len('/kaggle/input/data-food/external/'):]
        columns = ['file', 'label']
        target = pd.DataFrame(columns=columns)
        
        for filename in filenames:
            target.loc[len(target)] = [filename, idx]
        
        dictionary[idx] =  label
        sub_dataset = CustomImageDataset(target, dirname, transform = train_transforms)
        
        if first_first == True:
            train_data = sub_dataset
            first_first = False
        else:
            train_data = ConcatDataset([train_data, sub_dataset])
        
        idx = idx + 1
        
    first = True     

In [8]:
# show a sample
train_data[2]

(tensor([[[0.0549, 0.0510, 0.0510,  ..., 0.0118, 0.0118, 0.0118],
          [0.0549, 0.0510, 0.0510,  ..., 0.0118, 0.0118, 0.0118],
          [0.0549, 0.0510, 0.0510,  ..., 0.0118, 0.0118, 0.0118],
          ...,
          [0.1647, 0.1647, 0.1569,  ..., 0.0392, 0.0392, 0.0392],
          [0.1686, 0.1686, 0.1608,  ..., 0.0431, 0.0431, 0.0431],
          [0.1686, 0.1686, 0.1608,  ..., 0.0431, 0.0431, 0.0431]],
 
         [[0.0157, 0.0118, 0.0118,  ..., 0.0118, 0.0118, 0.0118],
          [0.0157, 0.0118, 0.0118,  ..., 0.0118, 0.0118, 0.0118],
          [0.0196, 0.0157, 0.0157,  ..., 0.0118, 0.0118, 0.0118],
          ...,
          [0.0353, 0.0353, 0.0392,  ..., 0.0235, 0.0235, 0.0235],
          [0.0392, 0.0392, 0.0431,  ..., 0.0196, 0.0196, 0.0196],
          [0.0392, 0.0392, 0.0431,  ..., 0.0196, 0.0196, 0.0196]],
 
         [[0.0118, 0.0078, 0.0078,  ..., 0.0039, 0.0039, 0.0039],
          [0.0118, 0.0078, 0.0078,  ..., 0.0039, 0.0039, 0.0039],
          [0.0118, 0.0078, 0.0078,  ...,

In [9]:
# split into training and validation set
train_percentage = 0.7
val_percentage = 0.3

total_length = len(train_data)

train_size = int(train_percentage * total_length)
val_size = total_length - train_size

train_subset, val_subset = torch.utils.data.random_split(train_data, [train_size, val_size], generator=torch.Generator().manual_seed(7767))

In [10]:
val_subset[14]

(tensor([[[0.4667, 0.4824, 0.4941,  ..., 0.2000, 0.2118, 0.2235],
          [0.4706, 0.4902, 0.4863,  ..., 0.2157, 0.2275, 0.2392],
          [0.4667, 0.4863, 0.4784,  ..., 0.2196, 0.2314, 0.2431],
          ...,
          [0.2510, 0.2784, 0.2980,  ..., 0.1137, 0.0941, 0.0980],
          [0.2039, 0.2314, 0.2392,  ..., 0.1333, 0.1294, 0.1255],
          [0.1961, 0.2196, 0.2275,  ..., 0.1059, 0.0980, 0.0863]],
 
         [[0.2588, 0.2745, 0.2902,  ..., 0.2627, 0.2745, 0.2863],
          [0.2549, 0.2824, 0.2824,  ..., 0.2784, 0.2902, 0.3020],
          [0.2471, 0.2745, 0.2706,  ..., 0.2784, 0.2941, 0.3059],
          ...,
          [0.3176, 0.3451, 0.3608,  ..., 0.0941, 0.0745, 0.0667],
          [0.2706, 0.2980, 0.3020,  ..., 0.1294, 0.1255, 0.1216],
          [0.2549, 0.2784, 0.2863,  ..., 0.1098, 0.1059, 0.0941]],
 
         [[0.1333, 0.1451, 0.1490,  ..., 0.0824, 0.0902, 0.1020],
          [0.1373, 0.1569, 0.1529,  ..., 0.0980, 0.1059, 0.1176],
          [0.1412, 0.1569, 0.1490,  ...,

### Data Loader 
Once the data are loaded in the workspace, we can prepare the data to feed them into the model.

In [11]:
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(
    train_subset, 
    batch_size=hparams["batch_size"], 
    shuffle=True,
    num_workers=1, 
    pin_memory=True,
)

val_loader = torch.utils.data.DataLoader(
    val_subset,
    batch_size=hparams["batch_size"],
    shuffle=False, 
    num_workers=1,
)

## Create pandas DataFrame from Tensor

In [29]:
# Initialize an empty DataFrame.
df = pd.DataFrame(columns=['size_dim1', 'size_dim2', 'size_dim3', 'min_colour_range', 'max_colour_range', 'type_torch.float32'])  

# Iterate through all tensors (images) in train_data and add size and range to the DataFrame
for i in tqdm(range(len(train_data)), desc="Processing"): 
    size = list(map(float, train_data[i][0].size()))  # Convert size to a list of floats.
    
    # Dynamic range of the colours in the image.
    min_colour_range = float(torch.min(train_data[i][0]))
    max_colour_range = float(torch.max(train_data[i][0]))

    data_type = train_data[i][0].dtype
    
    data_type = train_data[i][0].dtype
    all_elements_float32 = (data_type == torch.float32)  # Check if all elements are of type torch.float32 

    df = pd.concat([df, pd.DataFrame({'size_dim1': [size[0]], 'size_dim2': [size[1]], 'size_dim3': [size[2]], 'min_colour_range': [min_colour_range], 'max_colour_range': [max_colour_range], 'type_torch.float32': [all_elements_float32]})], ignore_index=True)

# Display the resulting DataFrame
print(df)

   size_dim1  size_dim2  size_dim3  min_colour_range  max_colour_range  \
0        3.0      224.0      224.0          0.000000          0.988235   
1        3.0      224.0      224.0          0.000000          1.000000   
2        3.0      224.0      224.0          0.000000          0.878431   
3        3.0      224.0      224.0          0.007843          1.000000   
4        3.0      224.0      224.0          0.000000          1.000000   
5        3.0      224.0      224.0          0.000000          0.960784   
6        3.0      224.0      224.0          0.000000          0.941176   
7        3.0      224.0      224.0          0.000000          0.886275   
8        3.0      224.0      224.0          0.000000          1.000000   
9        3.0      224.0      224.0          0.000000          0.913725   

  type_torch.float32  
0               True  
1               True  
2               True  
3               True  
4               True  
5               True  
6               True  
7

In [30]:
# unique values in the 'size_dim2' column
print(df['size_dim1'].value_counts())

3.0    10
Name: size_dim1, dtype: int64


In [31]:
print(df['size_dim2'].value_counts())

224.0    10
Name: size_dim2, dtype: int64


In [32]:
print(df['size_dim3'].value_counts())

224.0    10
Name: size_dim3, dtype: int64


In [33]:
# size of the dataset
print(df.shape)

(10, 6)


In [34]:
# get the index of the rows with size_dim1 = 1
indexNames = df[df['size_dim1'] == 1].index
print(indexNames)

# delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

Int64Index([], dtype='int64')


In [35]:
# size of the dataset
print(df.shape)

(10, 6)


In [36]:
df.to_csv('/Users/violeta/Desktop/gced/Q7/TAED2/project1/taed2-Food_Classification/data/processed/x_data_information.csv', index=False)

In [24]:
# Initialize an empty DataFrame.
y_df = pd.DataFrame(columns=['label_id'])  

# Iterate through all tensors (images) in train_data and add size and range to the DataFrame
for i in tqdm(range(len(train_data)), desc="Processing"): 
    label_id = train_data[i][1]

    y_df = pd.concat([y_df, pd.DataFrame({'label_id': [label_id]})], ignore_index=True)

# Display the resulting DataFrame
print(y_df)

  label_id
0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0


In [25]:
# delete these row indexes from dataFrame
y_df.drop(indexNames , inplace=True)

In [27]:
y_df.to_csv('/Users/violeta/Desktop/gced/Q7/TAED2/project1/taed2-Food_Classification/data/processed/y_data_information.csv', index=False)